In [32]:
import importlib

import numpy as np

import rough_path_signatures_pricing.signature_regressor as sr
import rough_path_signatures_pricing.simulation_models as sm

importlib.reload(sm)
importlib.reload(sr)

<module 'rough_path_signatures_pricing.signature_regressor' from 'C:\\Users\\chris\\VSCodeProjects\\rough-path-signatures-pricing\\src\\rough_path_signatures_pricing\\signature_regressor.py'>

#### GBM Paths for Fitting Linear Functional with Signature to Approximate Payoffs

In [33]:
gbm_params = {"x0": 10, "t0": 0, "t1": 1, "n_steps": 100, "n_paths": 100}
gbm_seed = 42

In [34]:
gbm = sm.GBM(mu=0, sigma=0.1, seed=gbm_seed)

In [35]:
gbm_paths, times, _ = gbm.simulate_paths(**gbm_params)
gbm_paths

array([[10.        , 10.03047171,  9.99254022, ..., 10.72244088,
        10.7212888 , 10.57930109],
       [10.        ,  9.89600159, 10.02457324, ..., 10.03525722,
         9.90067873,  9.92545507],
       [10.        , 10.07504512, 10.03915136, ...,  9.67202115,
         9.70943813,  9.81392868],
       ...,
       [10.        ,  9.90027532,  9.86120832, ..., 10.38210752,
        10.37781999, 10.38666695],
       [10.        , 10.03997742, 10.03946319, ...,  8.90181763,
         9.03778865,  9.13944065],
       [10.        ,  9.90945209,  9.8932558 , ...,  9.27859918,
         9.22764428,  9.21077154]], shape=(100, 101))

#### Fitting Linear Functional

In [36]:
N = 5


def asianPayoff(x: np.ndarray) -> np.ndarray:
    return np.mean(x, axis=1)

In [37]:
gbm_signature_price = sr.SignaturePricer(signature_degree=N, payoff=asianPayoff)
gbm_signature_price.fit(gbm_paths, times)

#### Creating Basket of Similar Options

In [38]:
hullwhite_params = {"x0": 10, "t0": 0, "t1": 1, "n_steps": 100, "n_paths": 25}
hullwhite_seed = 42


def theta(t):
    return 0.03

In [39]:
hullwhite = sm.HullWhite(a=1, sigma=0.1, theta=theta, seed=hullwhite_seed)

In [40]:
hullwhite_paths, _, _ = hullwhite.simulate_paths(**hullwhite_params)
payoffs = hullwhite_paths

In [41]:
prices = asianPayoff(payoffs)
prices

array([6.33057037, 6.40023932, 6.37595713, 6.32474915, 6.31353483,
       6.37463307, 6.30025867, 6.31200737, 6.28925178, 6.26430336,
       6.29827663, 6.26328668, 6.29215938, 6.26486319, 6.35008527,
       6.27759942, 6.33549516, 6.34921802, 6.31743636, 6.26255873,
       6.26848975, 6.31656223, 6.28547231, 6.32479423, 6.25131481])

In [42]:
gbm_signature_price.price(prices)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional